## Import packages

In [ ]:
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm

## Load data

In [ ]:
train_data = joblib.load('../input/train-data-with-pred/train_data_with_model_pred_and_performance.pkl')

## Modelling

In [ ]:
train_data.head(10)

In [ ]:
train_data['valid pred flag'] = np.where(train_data['F1 score']!=0, 1, 0)
master_data = train_data[train_data['Prediction'].apply(lambda x: len(x)>0)][['annotation','Prediction','feature_text','valid pred flag']].copy()

In [ ]:
print(master_data.shape)
print('Valid preds:', master_data['valid pred flag'].sum())
master_data.head(10)

In [ ]:
master_data_pivoted = pd.DataFrame(data = None, columns = ['Text', 'Feature text','Correct pred flag'])
count = 0
for i in tqdm(range(0, len(master_data))):
    for j in master_data['Prediction'].iloc[i]:
        text = j[0]
        master_data_pivoted.loc[count] = text, master_data['feature_text'].iloc[i], master_data['valid pred flag'].iloc[i]
        count = count + 1

In [ ]:
master_data_pivoted['Feature text'] = master_data_pivoted['Feature text'].apply(lambda x: x.replace('-',' '))
print(master_data_pivoted.shape)
print('Correct pred %:', master_data_pivoted['Correct pred flag'].sum()/master_data_pivoted.shape[0] * 100)
master_data_pivoted.head(10)

In [ ]:
master_data_pivoted[master_data_pivoted['Correct pred flag']==0]

In [ ]:
print('# Feature categories:', master_data_pivoted[master_data_pivoted['Correct pred flag']==0]['Feature text'].nunique())
master_data_pivoted[master_data_pivoted['Correct pred flag']==0]['Feature text'].value_counts()

In [ ]:
feature_text = 'Subjective fever'
print('\nIncorrect preds:\n')
display(master_data_pivoted[(master_data_pivoted['Feature text']==feature_text)  & (master_data_pivoted['Correct pred flag']==0)]['Text'].unique())
print('\nCorrect preds:\n')
display(master_data_pivoted[(master_data_pivoted['Feature text']==feature_text)  & (master_data_pivoted['Correct pred flag']==1)]['Text'].unique())

In [ ]:
joblib.dump(master_data_pivoted, 'train_data_error_analysis.pkl')